In [ ]:
from datasets import load_dataset

# 載入 ChnSentiCorp 中文情感數據集
dataset = load_dataset("")
train_data = dataset["train"]
test_data = dataset["test"]

Downloading: 100%|██████████| 1.88k/1.88k [00:00<00:00, 3.44MB/s]
Using custom data configuration default


FileNotFoundError: Couldn't find file at https://drive.google.com/u/0/uc?id=1uV-aDQoMI51A27OxVgJnzxqZFQqkDydZ&export=download

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 二分類模型

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

encoded_train = train_data.map(preprocess_function, batched=True)
encoded_test = test_data.map(preprocess_function, batched=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
)

# 開始訓練
trainer.train()

In [ ]:
# 測試文本
text = "這部電影真的很好看！"

# 將文本轉換為模型輸入格式
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

# 預測
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1).item()

# 輸出情感結果
if predictions == 1:
    print("情感：正面")
else:
    print("情感：負面")